In [ ]:
library(Seurat, lib = "/frazer01/home/mdonovan/alt_r_library/")
library(dplyr)
library(biomaRt)
library(stringr)


In [2]:
setwd("/projects/PPC/analysis/ppc_pilot/data/")
dir.create("qc", showWarnings = FALSE)
dir.create("robjs", showWarnings = FALSE)
dir.create("UMAP", showWarnings = FALSE)
dir.create("differential_expression", showWarnings = FALSE)
dir.create("gene_expression", showWarnings = FALSE)
dir.create("integrated", showWarnings = FALSE)
dir.create("prediction", showWarnings = FALSE)

marker_genes <- c("INS", "NKX6-1", "NPTX2", "ITGA1", "PCSK1",
                         "GCG", "DPP4",   "ARX",   "IRX2",  "ETV1",
                         "TPH1", "SLC18A1", "LMX1A", "ADRA2A", "MME",
                         "CHGA", "FEV", "SYP", "CPE", "CDKN1C",
                         "SOX9", "PTF1A", "PDX1",
                         "TOP2A", "AURKB", "ISL1", "PAX4", "NEUROG3")

In [3]:
#### load melvin data #####
load("/projects/PPC/analysis/ppc_pilot/data/melvin_data/Stages_3_to_6.x1_S3c.Robj")
S3c = tiss
load("/projects/PPC/analysis/ppc_pilot/data/melvin_data/Stages_3_to_6.x1_S4c.Robj")
S4c = tiss
load("/projects/PPC/analysis/ppc_pilot/data/melvin_data/Stages_3_to_6.x1_S5c.Robj")
S5c = tiss
load("/projects/PPC/analysis/ppc_pilot/data/melvin_data/Stages_3_to_6.x1_S6c.Robj")
S6c = tiss
load("/projects/PPC/analysis/ppc_pilot/data/melvin_data/ESB_only_integrated.Robj")
scb_integrated = integrated

S3c@meta.data$Celltype <- S3c@meta.data$Assigned_cluster
S4c@meta.data$Celltype <- S4c@meta.data$Assigned_cluster
S5c@meta.data$Celltype <- S5c@meta.data$Assigned_cluster
S6c@meta.data$Celltype <- S6c@meta.data$Assigned_cluster
S3c@meta.data$set    <- S3c@meta.data$Lib_prep_batch
S4c@meta.data$set    <- S4c@meta.data$Lib_prep_batch
S5c@meta.data$set    <- S5c@meta.data$Lib_prep_batch
S6c@meta.data$set    <- S6c@meta.data$Lib_prep_batch
###########################

In [4]:
meta = read.csv("/projects/PPC/analysis/ppc_pilot/tables/ppc_pilot_metadata.csv")

In [5]:
meta

udid,name,type,path
<chr>,<chr>,<chr>,<chr>
PPC023,PPC023_PPC_fresh,PPC,/projects/PPC/pipeline/scRNA-Seq/pp_190304/fb6028d0-5ef0-4f53-bf73-da3983948ce1/outs/filtered_feature_bc_matrix/
PPC027,PPC027_PPC_fresh,PPC,/projects/PPC/pipeline/scRNA-Seq/pp_190304/7d700751-e80d-40b6-9e79-0efec5ae5a80/outs/filtered_feature_bc_matrix/
PPC028,PPC028_PPC_fresh,PPC,/projects/PPC/pipeline/scRNA-Seq/pp_190304/e2308d3d-f183-44e4-b0b7-2dff4a5006da/outs/filtered_feature_bc_matrix/
PPC029,PPC029_PPC_fresh,PPC,/projects/PPC/pipeline/scRNA-Seq/pp_190304/4efdab78-1eda-4888-8483-15b519c375dc/outs/filtered_feature_bc_matrix/
PPC030,PPC030_PPC_fresh,PPC,/projects/PPC/pipeline/scRNA-Seq/pp_190304/4fd8f0f0-2356-451c-a61f-c30610f1141c/outs/filtered_feature_bc_matrix/
PPC034,PPC034_PPC_fresh,PPC,/projects/PPC/pipeline/scRNA-Seq/pp_190304/3ab5e63d-1d2d-441b-b1c3-7f77fd86e4dd/outs/filtered_feature_bc_matrix/
PPC036,PPC036_PPC_fresh,PPC,/projects/PPC/pipeline/scRNA-Seq/pp_190304/eada4dfd-14cc-4767-8791-4a478eb3fe12/outs/filtered_feature_bc_matrix/
PPC034,PPC034_iPSC_fresh,iPSC,/projects/PPC/pipeline/scRNA-Seq/pp_190304/1eeda2ea-453a-4ff4-9cae-2a68f823d919/outs/filtered_feature_bc_matrix/
PPCfreshmerged,PPC_fresh_merged,PPC,/projects/PPC/pipeline/scRNA-Seq/ag_190326/ppc/outs/filtered_feature_bc_matrix/


In [7]:
temp_meta = data.frame(
                        udid = "Frazermerged_new",
                        name = "Frazermerged_new",
                        type = "All",
                        path = "/projects/PPC/pipeline/scRNA-Seq/ag_190708/ppc/outs/filtered_feature_bc_matrix/"
          )

In [12]:
for (m in seq(1, nrow(temp_meta))){
    
    sample_name = temp_meta[m, "name"]
    path        = temp_meta[m, "path"]
    
    
    print(sample_name)
    
    tiss <- Read10X(data.dir = path)
    tiss <- CreateSeuratObject(counts = tiss, project = sample_name)
    
    tiss[["percent.mt"]] <- PercentageFeatureSet(tiss, pattern = "^MT")
    
    png(file = paste("./qc/", sample_name, "_umi.vs.mt.png",sep = ""), width = 5, height = 5, units = "in", res = 300)
    print(plot(tiss@meta.data$nCount_RNA, tiss@meta.data$percent.mt, pch = 20, cex = .5, xlab = "nUMI", ylab = "%MT", las = 1))
    abline(h = 10, col = "red")
    dev.off()
    
    png(file = paste("./qc/", sample_name, "_umi.vs.genes.png",sep = ""), width = 5, height = 5, units = "in", res = 300)
    print(plot(tiss@meta.data$nCount_RNA, tiss@meta.data$nFeature_RNA, pch = 20, cex = .5, xlab = "nUMI", ylab = "n Unique Genes", las = 1) )
    abline(h = 500, col = "red")
    abline(h = 6000, col = "red")
    dev.off()
    
    print("saving unfiltered robj")
    save(tiss, file = paste("./robjs/", sample_name, "_seurat_unfiltered.robj", sep = ""))

    print("processing data")
    tiss <- subset(tiss, subset = nFeature_RNA > 500 & nFeature_RNA < 6000 & percent.mt < 10)
    tiss <- NormalizeData(tiss)
    tiss <- FindVariableFeatures(tiss, selection.method = "vst", nfeatures = 2000)
    all.genes <- rownames(tiss)
    tiss <- ScaleData(tiss)
    
    tiss <- RunPCA(tiss, features = VariableFeatures(object = tiss))
    
    pdf(paste("./qc/", sample_name, "_elbow_plot.pdf",sep = ""), height = 5, width = 10)
    print(ElbowPlot(tiss, ndims = 50))
    dev.off()
    
    n.pcs = 20
    res.used <- .5
    
    print("clustering data")
    tiss <- FindNeighbors(tiss, dims = 1:n.pcs)
    tiss <- FindClusters( tiss, resolution = res.used)
    tiss <- RunUMAP(tiss, dims = 1:n.pcs, min_dist = .8, seed = 10)
    
    png(file = paste("./UMAP/", sample_name, "_cluster_UMAP.png",sep = ""), width = 10, height = 5, units = "in", res = 300)
    print(DimPlot(tiss, reduction.use = "umap", label = TRUE, pt.size = 1,
            vector.friendly = T, png.file = paste("./UMAP/", sample_name, "_cluster_UMAP.png",sep = ""),
            png.arguments = c(10, 5, 300), do.return=TRUE))
    dev.off()
    
    pdf(paste("./gene_expression/", sample_name, "_heatmap.pdf",sep = ""), height = 5, width = 10)
    print(DoHeatmap(tiss, features = marker_genes) + NoLegend())
    dev.off()
    
    pdf(paste("./gene_expression/", sample_name, "_heatmap.pdf",sep = ""), height = 5, width = 10)
    print(FeaturePlot(object = tiss, features = toupper(unique(marker_genes))))
    dev.off()
    
    clusters = seq(0, (length(unique(tiss@meta.data$RNA_snn_res.0.5)) - 1))
    for( i in clusters){
        print(paste("finding DE genes in cluster"), i, sep = " ")
        clusterX.markers <- FindMarkers(tiss, ident.1 = i, min.pct = 0.25)
        write.csv(clusterX.markers, file = paste("./differential_expression/", sample_name, "_cluster_", i, ".csv", sep = ""))
    }
    
    print("saving processed data")
    save(tiss, file = paste("./robjs/", sample_name, "_seurat_filtered.robj",sep = ""))
    
    
    ### integrate
    tiss@meta.data$set <- sample_name
    tiss@meta.data$Celltype <- tiss@meta.data$RNA_snn_res.0.5
    
    data2integrate = list()
    data2integrate[["tiss"]] = tiss
    data2integrate[["S3c"]]    = S3c
    data2integrate[["S4c"]]    = S4c
    data2integrate[["S5c"]]    = S5c
    data2integrate[["S6c"]]    = S6c
    
    print("integrating data")
    anchors    <- FindIntegrationAnchors(object.list = data2integrate, dims = 1:30)
    integrated <- IntegrateData(anchorset = anchors, dims = 1:30)
    
    DefaultAssay(integrated) <- "integrated"
    
    integrated <- ScaleData(integrated, verbose = FALSE)
    integrated <- RunPCA(   integrated, npcs = 30, verbose = FALSE)
    integrated <- RunUMAP(  integrated, reduction = "pca", dims = 1:30, seed = 10, min.dist = 0.1, n.neighbors = 30)
    
    print("saving integrated data")
    save(integrated, file = paste("./robjs/", sample_name, "_seurat_SCBintegrated.robj",sep = ""))   
    
    png(file = paste("./UMAP/", sample_name, "_SCBintegrated_byset_UMAP.png",sep = ""), width = 10, height = 5, units = "in", res = 300)
    print(DimPlot(integrated, group.by = "set", label = T, pt.size = .15,
            vector.friendly = T, png.file = paste("./UMAP/", sample_name, "_SCBintegrated_byset_UMAP.png",sep = ""),
            png.arguments = c(10, 5, 300), do.return=TRUE))
    dev.off()
    
    png(file = paste("./UMAP/", sample_name, "_SCBintegrated_bycelltype_UMAP.png",sep = ""), width = 10, height = 5, units = "in", res = 300)
    print(DimPlot(integrated, group.by = "Celltype", label = T, pt.size = .15,
            vector.friendly = T, png.file = paste("./UMAP/", sample_name, "_SCBintegrated_bycelltype_UMAP.png",sep = ""),
            png.arguments = c(10, 5, 300), do.return=TRUE))
    dev.off()
    
    
    ######project########
    print("projecting data")
    query = tiss
    anchors     <- FindTransferAnchors(reference = scb_integrated, query = query, dims = 1:30)
    predictions <- TransferData(anchorset = anchors, refdata = scb_integrated$Standard_celltype, dims = 1:30)
    query       <- AddMetaData(query, metadata = predictions)

    print("saving projected data")
    save(query, file = paste("./robjs/", sample_name, "_seurat_SCBprojection.robj",sep = ""))    
    
    png(file = paste("./UMAP/", sample_name, "_SCB_projected_UMAP.png",sep = ""), width = 10, height = 5, units = "in", res = 300)
    print(DimPlot(query, group.by = "predicted.id", label = T,
            vector.friendly = T, png.file = paste("./UMAP/", sample_name, "_SCB_projected_UMAP.png",sep = ""),
            png.arguments = c(10, 5, 300), do.return=TRUE))
    dev.off()
    
    ############
    
    tiss       = NULL
    integrated = NULL
    query      = NULL
    
}

[1] "Frazermerged_new"


Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”

NULL
NULL
[1] "saving unfiltered robj"
[1] "processing data"


Centering and scaling data matrix


PC_ 1 
Positive:  SPINK1, NEAT1, KRT8, AMBP, H19, S100A6, TMSB10, FN1, ANXA2, DLK1 
	   SERPINA1, SAT1, VTN, IGFBP2, KRT18, CDH6, ATP1B1, KRT19, IGFBP7, IGF2 
	   CCND1, S100A10, RBP1, S100A11, CST3, KCNQ1OT1, DSP, CDKN1C, C12orf75, CD59 
Negative:  POU5F1, MIR302B, TDGF1, ESRG, SFRP2, DNMT3B, DPPA4, L1TD1, PSAT1, THY1 
	   RP4-792G4.2, TERF1, ZIC2, POU5F1B, FABP5, PODXL, VRTN, LECT1, POLR3G, LINC00678 
	   UCHL1, H2AFZ, SEPHS1, SOX2, ZSCAN10, SFRP1, SCGB3A2, UBE2C, USP44, CBS 
PC_ 2 
Positive:  KCNQ1OT1, CALB1, EDN3, CDKN1C, AMBP, DCX, SCGN, TXNIP, LINC00261, PDX1 
	   IGF2, SULT1E1, TMEM176B, TMEM176A, FRZB, CA4, ALDH1A1, NKX6-1, H19, DKK2 
	   AOC1, PPP1R1A, CADPS, PKDCC, SOX21, SLC4A4, RP11-834C11.4, GFRA3, GSTA1, FXYD3 
Negative:  TAGLN, AKAP12, TM4SF1, ACTB, IL32, PLAU, TNFRSF12A, VIM, TFPI2, TPM1 
	   PRSS23, PHLDA2, MYL9, S100A6, NPPB, PFN1, TUBA1A, PMEPA1, IL11, LTB 
	   TUBB6, TPM4, GDF15, MYL12A, TAGLN2, ITGAV, COTL1, TUBA1C, GSTP1, TUBA1B 
PC_ 3 
Positive:  THY1, POU5F1, GA

...

[1] "finding DE genes in cluster"
[1] "finding DE genes in cluster"
[1] "finding DE genes in cluster"
[1] "finding DE genes in cluster"
[1] "finding DE genes in cluster"
[1] "finding DE genes in cluster"
[1] "finding DE genes in cluster"
[1] "finding DE genes in cluster"
[1] "finding DE genes in cluster"
[1] "finding DE genes in cluster"
[1] "finding DE genes in cluster"
[1] "finding DE genes in cluster"
[1] "finding DE genes in cluster"
[1] "finding DE genes in cluster"
[1] "finding DE genes in cluster"
[1] "finding DE genes in cluster"
[1] "finding DE genes in cluster"
[1] "saving processed data"
[1] "integrating data"


Computing 2000 integration features
Scaling features for provided objects
Finding all pairwise anchors
Running CCA
Merging objects
Finding neighborhoods
Finding anchors
	Found 36900 anchors
Filtering anchors
	Retained 3011 anchors
Extracting within-dataset neighbors
Running CCA
Merging objects
Finding neighborhoods
Finding anchors
	Found 29874 anchors
Filtering anchors
	Retained 1929 anchors
Extracting within-dataset neighbors
Running CCA
Merging objects
Finding neighborhoods
Finding anchors
	Found 22475 anchors
Filtering anchors
	Retained 1118 anchors
Extracting within-dataset neighbors
Running CCA
Merging objects
Finding neighborhoods
Finding anchors
	Found 19307 anchors
Filtering anchors
	Retained 1743 anchors
Extracting within-dataset neighbors
Running CCA
Merging objects
Finding neighborhoods
Finding anchors
	Found 15066 anchors
Filtering anchors
	Retained 951 anchors
Extracting within-dataset neighbors
Running CCA
Merging objects
Finding neighborhoods
Finding anchors
	Found 12944

[1] "saving integrated data"
[1] "projecting data"


Performing PCA on the provided reference using 2000 features as input.
Projecting PCA
Finding neighborhoods
Finding anchors
	Found 2299 anchors
Filtering anchors
	Retained 337 anchors
Extracting within-dataset neighbors


ERROR: Error in SNNAnchor(k_matrix = k.matrix, anchor_only = anchor.only): std::bad_alloc


In [15]:
tiss@meta.data

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,RNA_snn_res.0.5,seurat_clusters,set,Celltype
,<fct>,<dbl>,<int>,<dbl>,<fct>,<fct>,<chr>,<fct>
AAACCCAAGACCATGG-1,Frazermerged_new,5831,2369,6.2253473,4,4,Frazermerged_new,4
AAACCCAAGGGTTAAT-1,Frazermerged_new,3887,1826,5.6856187,4,4,Frazermerged_new,4
AAACCCACAACCAACT-1,Frazermerged_new,4812,2100,5.1953450,4,4,Frazermerged_new,4
AAACCCACACACTGGC-1,Frazermerged_new,10110,3453,5.5093966,4,4,Frazermerged_new,4
AAACCCACACCGTGAC-1,Frazermerged_new,7399,2833,5.4331666,4,4,Frazermerged_new,4
AAACCCACAGCACACC-1,Frazermerged_new,3533,1739,5.4627795,4,4,Frazermerged_new,4
AAACCCAGTCTGCCTT-1,Frazermerged_new,5928,2455,4.5209177,4,4,Frazermerged_new,4
AAACCCAGTGGTCTCG-1,Frazermerged_new,5174,2204,4.4066486,4,4,Frazermerged_new,4
AAACCCAGTTCGAGCC-1,Frazermerged_new,4483,2059,5.6881553,4,4,Frazermerged_new,4
